### Import libraries

In [67]:
from stockfish import Stockfish
from stockfish import StockfishException
import pandas as pd
import numpy as np
import re
import chess
import random

### Import dataset

In [68]:
with open('posiciones.txt') as f:
    pos = f.readlines()
pos1 = []
for i in range(len(pos)):
    pos1.append(re.sub('\n','', pos[i]))    
    

We duplicate each position: In first, blacks make the next move and in the second, whites make it 

In [69]:
bl = []
wh = []
for i in range(len(pos1)):
    bl.append(pos1[i] + ' b')
    wh.append(pos1[i] + ' w')
dataset = bl + wh

### Additional fucntions

In [70]:
def evaluation(pos, elo, depth):
    stockfish = Stockfish('stockfish_15_x64_avx2.exe')
    stockfish.set_elo_rating(elo)        
    stockfish.set_depth(depth)
    stockfish.set_fen_position(pos)
    ev = stockfish.get_evaluation()
    pos = stockfish.get_top_moves(1)
    return ev, pos

In [71]:
def dif1(val1,val2):
    if abs(val1)+abs(val2) == 0:
        return 0
    else:
        return abs(val1-val2)/(abs(val1)+abs(val2))
def dif2(val1,val2):
    return float(abs(val1-val2))

In [72]:
def MR_equi(o1,o2,delta,epsilon):
    final = bool()
    if o1.get('type') != o2.get('type'):
        final = False
    elif o1.get('type') == o2.get('type') == 'mate' and o1.get('value') != o2.get('value'):
        final = False
    elif o1.get('type') == o2.get('type') == 'cp' and dif1(o1.get('value'),o2.get('value')) > delta and dif2(o1.get('value'),o2.get('value')) >= epsilon:
        final = False
    else:
        final = True
    return final

In [73]:
def sim_axis(pos):
    pos_transformed = []
    col = pos[-1:]
    aux1 = pos[:-2]
    aux2 = aux1.split("/")
    aux3 = []
    for j in range(8):
        aux3.append(aux2[j][::-1])
    aux4 = '/'.join(aux3)
    pos_transformed.append(aux4)
    if col == 'w':
        pos_transformed = pos_transformed[0] + ' w'
    if col == 'b':
        pos_transformed = pos_transformed[0] + ' b'
    return pos_transformed

In [74]:
def checkcheckW(pos):
    def make_matrix(board): #type(board) == chess.Board()
        pgn = board
        foo = []  #Final board
        pieces = pgn.split(" ", 1)[0]
        rows = pieces.split("/")
        for row in rows:
            foo2 = []  #This is the row I make
            for thing in row:
                if thing.isdigit():
                    for i in range(0, int(thing)):
                        foo2.append('.')
                else:
                    foo2.append(thing)
            foo.append(foo2)
        return foo
    whites = ['Q','K','R','B','N','P']
    blacks = ['q', 'k', 'r', 'b', 'n', 'p']
    a = make_matrix(pos)
    check = bool()
    pos = []
    for i in range(8):
        for j in range(8):
            if a[i][j] == 'K':
                pos.append(i)
                pos.append(j)
    for i in list(reversed(range(pos[1]))):
        if a[pos[0]][i] in whites or a[pos[0]][i] in list(blacks[j] for j in [1,3,4,5]):
            break
        elif a[pos[0]][i] == 'q' or a[pos[0]][i]== 'r':
            check = True
            break
    for i in list(range(pos[1]+1,8)):
        if a[pos[0]][i] in whites or a[pos[0]][i] in list(blacks[j] for j in [1,3,4,5]):
            break
        elif a[pos[0]][i] == 'q' or a[pos[0]][i]== 'r':
            check = True
            break
    for i in list(reversed(range(pos[0]))):
        if a[i][pos[1]] in whites or a[i][pos[1]] in list(blacks[j] for j in [1,3,4,5]):
            break
        elif a[i][pos[1]] == 'q' or a[i][pos[1]]== 'r':
            check = True
            break
    for i in list(range(pos[0]+1,8)):
        if a[i][pos[1]] in whites or a[i][pos[1]] in list(blacks[j] for j in [1,3,4,5]):
            break
        elif a[i][pos[1]] == 'q' or a[i][pos[1]]== 'r':
            check = True
            break
    r = 8 - np.maximum(abs(pos[0]), abs(pos[1]))
    r1 = np.minimum(abs(pos[0]), abs(pos[1]))
    r2 = np.minimum(r,r1)
    for i in list(range(1,r)):
        if a[pos[0]+i][pos[1]+i] in whites or a[pos[0]+i][pos[1]+i] in list(blacks[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]+i][pos[1]+i] == 'b' or a[pos[0]+i][pos[1]+i] == 'q':
            check = True
            break
    for i in list(range(1,r2)):
        if a[pos[0]-i][pos[1]+i] in whites or a[pos[0]-i][pos[1]+i] in list(blacks[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]-i][pos[1]+i] == 'b' or a[pos[0]-i][pos[1]+i] == 'q':
            check = True
            break
    for i in list(range(1,r2)):
        if a[pos[0]-i][pos[1]-i] in whites or a[pos[0]-i][pos[1]-i] in list(blacks[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]-i][pos[1]-i] == 'b' or a[pos[0]-i][pos[1]-i] == 'q':
            check = True
            break
    for i in list(range(1,r)):
        if a[pos[0]+i][pos[1]-i] in whites or a[pos[0]+i][pos[1]-i] in list(blacks[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]+i][pos[1]-i] == 'b' or a[pos[0]+i][pos[1]-i] == 'q':
            check = True
            break
    if pos[0]+2 < 8 and pos[1]-1 > 0 and a[pos[0]+2][pos[1]-1] == 'n':
        check = True
    if pos[0]+2 < 8 and pos[1]+1 < 8 and a[pos[0]+2][pos[1]+1] == 'n':
        check = True
    if pos[0]-2 > 0 and pos[1]-1 > 0 and a[pos[0]-2][pos[1]-1] == 'n':
        check = True
    if pos[0]-2 > 0 and pos[1]+1 < 8 and a[pos[0]-2][pos[1]+1] == 'n':
        check = True
    if pos[0]+1 < 8 and pos[1]-2 > 0 and a[pos[0]+1][pos[1]-2] == 'n':
        check = True
    if pos[0]+1 < 8 and pos[1]+2 < 8 and a[pos[0]+1][pos[1]+2] == 'n':
        check = True
    if pos[0]-1 > 0 and pos[1]-2 > 0 and a[pos[0]-1][pos[1]-2] == 'n':
        check = True
    if pos[0]-1 > 0 and pos[1]+2 < 8 and a[pos[0]-1][pos[1]+2] == 'n':
        check = True
    if pos[0]-1 > 0 and pos[1]-1 > 0 and a[pos[0]-1][pos[1]-1] == 'p':
        check = True
    if pos[0]-1 > 0 and pos[1]+1 < 8 and a[pos[0]-1][pos[1]+1] == 'p':
        check = True
    return check

In [75]:
def checkcheckB(pos):
    def make_matrix(board): #type(board) == chess.Board()
        pgn = board
        foo = []  #Final board
        pieces = pgn.split(" ", 1)[0]
        rows = pieces.split("/")
        for row in rows:
            foo2 = []  #This is the row I make
            for thing in row:
                if thing.isdigit():
                    for i in range(0, int(thing)):
                        foo2.append('.')
                else:
                    foo2.append(thing)
            foo.append(foo2)
        return foo
    whites = ['Q','K','R','B','N','P']
    blacks = ['q', 'k', 'r', 'b', 'n', 'p']
    a = make_matrix(pos)
    check = bool()
    pos = []
    for i in range(8):
        for j in range(8):
            if a[i][j] == 'k':
                pos.append(i)
                pos.append(j)
    for i in list(reversed(range(pos[1]))):
        if a[pos[0]][i] in blacks or a[pos[0]][i] in list(whites[j] for j in [1,3,4,5]):
            break
        elif a[pos[0]][i] == 'Q' or a[pos[0]][i]== 'R':
            check = True
            break
    for i in list(range(pos[1]+1,8)):
        if a[pos[0]][i] in blacks or a[pos[0]][i] in list(whites[j] for j in [1,3,4,5]):
            break
        elif a[pos[0]][i] == 'Q' or a[pos[0]][i]== 'R':
            check = True
            break
    for i in list(reversed(range(pos[0]))):
        if a[i][pos[1]] in blacks or a[i][pos[1]] in list(whites[j] for j in [1,3,4,5]):
            break
        elif a[i][pos[1]] == 'Q' or a[i][pos[1]]== 'R':
            check = True
            break
    for i in list(range(pos[0]+1,8)):
        if a[i][pos[1]] in blacks or a[i][pos[1]] in list(whites[j] for j in [1,3,4,5]):
            break
        elif a[i][pos[1]] == 'Q' or a[i][pos[1]]== 'R':
            check = True
            break
    r = 8 - np.maximum(abs(pos[0]), abs(pos[1]))
    r1 = np.minimum(abs(pos[0]), abs(pos[1]))
    r2 = np.minimum(r,r1)
    for i in list(range(1,r)):
        if a[pos[0]+i][pos[1]+i] in blacks or a[pos[0]+i][pos[1]+i] in list(whites[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]+i][pos[1]+i] == 'B' or a[pos[0]+i][pos[1]+i] == 'Q':
            check = True
            break
    for i in list(range(1,r2)):
        if a[pos[0]-i][pos[1]+i] in blacks or a[pos[0]-i][pos[1]+i] in list(whites[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]-i][pos[1]+i] == 'B' or a[pos[0]-i][pos[1]+i] == 'Q':
            check = True
            break
    for i in list(range(1,r2)):
        if a[pos[0]-i][pos[1]-i] in blacks or a[pos[0]-i][pos[1]-i] in list(whites[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]-i][pos[1]-i] == 'B' or a[pos[0]-i][pos[1]-i] == 'Q':
            check = True
            break
    for i in list(range(1,r)):
        if a[pos[0]+i][pos[1]-i] in blacks or a[pos[0]+i][pos[1]-i] in list(whites[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]+i][pos[1]-i] == 'B' or a[pos[0]+i][pos[1]-i] == 'Q':
            check = True
            break
    if pos[0]+2 < 8 and pos[1]-1 > 0 and a[pos[0]+2][pos[1]-1] == 'N':
        check = True
    if pos[0]+2 < 8 and pos[1]+1 < 8 and a[pos[0]+2][pos[1]+1] == 'N':
        check = True
    if pos[0]-2 > 0 and pos[1]-1 > 0 and a[pos[0]-2][pos[1]-1] == 'N':
        check = True
    if pos[0]-2 > 0 and pos[1]+1 < 8 and a[pos[0]-2][pos[1]+1] == 'N':
        check = True
    if pos[0]+1 < 8 and pos[1]-2 > 0 and a[pos[0]+1][pos[1]-2] == 'N':
        check = True
    if pos[0]+1 < 8 and pos[1]+2 < 8 and a[pos[0]+1][pos[1]+2] == 'N':
        check = True
    if pos[0]-1 > 0 and pos[1]-2 > 0 and a[pos[0]-1][pos[1]-2] == 'N':
        check = True
    if pos[0]-1 > 0 and pos[1]+2 < 8 and a[pos[0]-1][pos[1]+2] == 'N':
        check = True
    if pos[0]-1 > 0 and pos[1]-1 > 0 and a[pos[0]-1][pos[1]-1] == 'P':
        check = True
    if pos[0]-1 > 0 and pos[1]+1 < 8 and a[pos[0]-1][pos[1]+1] == 'P':
        check = True
    return check

In [76]:
def bothcheck(pos):
    both = bool()
    if checkcheckB(pos) == True and checkcheckW(pos) == True:
        both = True
    return both

### Main function 

In [83]:
def main(dataset, elo, depth, delta, epsilon):
    pos1 = []
    pos2 = []
    results1 = []
    results2 = []
    MR = []
    index = []
    for i in range(len(dataset)):
        if bothcheck(dataset[i]) == False and bothcheck(sim_axis(dataset[i])) == False:
            if i%5==0: 
                print(i)
            try:
                MR.append(MR_equi(evaluation(dataset[i],elo,depth)[0],evaluation(sim_axis(dataset[i]), elo, depth)[0], delta, epsilon)) ## adding MR to dataframe
                pos1.append(dataset[i]) ## adding original position to dataframe
                pos2.append(sim_axis(dataset[i])) ## adding transformed position to dataframe
                results1.append(evaluation(dataset[i], elo, depth)[0]) ## adding evaluation of original position to dataframe
                results2.append(evaluation(sim_axis(dataset[i]), elo, depth)[0]) ## adding evaluation of transformed position to dataframe
                index.append(i) ## adding index to dataframe
            except StockfishException:
                pass
    d = {'index' : index, 'pos1' : pos1, 'evaluation pos1' : results1, 'pos2' : pos2, 'evaluation pos2' : results2,
        'MR' : MR}
    df = pd.DataFrame(data=d) ## creating dataframe
    return df

### Application of main function 

An example with the first 40 positions of our dataset. 

In [82]:
main(dataset[0:40], 4000, 10, 0.05, 10)

0
5
15
20
25
30
35


,index,pos1,evaluation pos1,pos2,evaluation pos2,MR
0,0,1b1B1Qr1/7p/6r1/2P5/4Rk2/1K6/4B3/8 b,"{'type': 'cp', 'value': 762}",1rQ1B1b1/p7/1r6/5P2/2kR4/6K1/3B4/8 b,"{'type': 'cp', 'value': 720}",True
1,1,1B1B2K1/1B6/5N2/6k1/8/8/8/4nq2 b,"{'type': 'cp', 'value': 0}",1K2B1B1/6B1/2N5/1k6/8/8/8/2qn4 b,"{'type': 'cp', 'value': 0}",True
2,5,1b1b4/1N5r/1P1p3B/8/3KP3/3P2k1/3PB3/1N6 b,"{'type': 'cp', 'value': -3}",4b1b1/r5N1/B3p1P1/8/3PK3/1k2P3/3BP3/6N1 b,"{'type': 'cp', 'value': 0}",True
3,7,1b1B4/1P6/2B5/3r2kN/2r4P/8/5K2/3B4 b,"{'type': 'cp', 'value': 790}",4B1b1/6P1/5B2/Nk2r3/P4r2/8/2K5/4B3 b,"{'type': 'cp', 'value': 901}",False
4,8,1B1b4/1Q2b3/8/6rn/3NK3/7k/2r5/8 b,"{'type': 'cp', 'value': -129}",4b1B1/3b2Q1/8/nr6/3KN3/k7/5r2/8 b,"{'type': 'cp', 'value': -142}",True
5,9,1b1B4/2b3PK/7P/8/1BP4k/4P3/8/8 b,"{'type': 'cp', 'value': 1308}",4B1b1/KP3b2/P7/8/k4PB1/3P4/8/8 b,"{'type': 'cp', 'value': 1316}",True
6,12,1B1B4/4R3/2R2p2/K7/8/3b3r/2Q4P/R2k4 b,"{'type': 'mate', 'value': 0}",4B1B1/3R4/2p2R2/7K/8/r3b3/P4Q2/4k2R b,"{'type': 'mate', 'value': 0}",True
7,13,1B1b4/6P1/P2BK3/8/5PR1/nPr5/N4k1P/8 b,"{'type': 'cp', 'value': 2125}",4b1B1/1P6/3KB2P/8/1RP5/5rPn/P1k4N/8 b,"{'type': 'cp', 'value': 2132}",True
8,14,1B1b4/8/8/2B5/3b4/1B2p1K1/8/6k1 b,"{'type': 'cp', 'value': -12}",4b1B1/8/8/5B2/4b3/1K1p2B1/8/1k6 b,"{'type': 'cp', 'value': -7}",True
9,15,1b1B4/8/K3N3/b1B5/8/k4P2/8/3n1B2 b,"{'type': 'cp', 'value': 518}",4B1b1/8/3N3K/5B1b/8/2P4k/8/2B1n3 b,"{'type': 'cp', 'value': 547}",True
